# Distilbert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers==4.30

In [ ]:
#!pip install datasets
#!pip install transformers
#!pip install torch

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_2000_balanced.csv')

In [ ]:
dataset.head()

,author_id,processed_text
0,132,man refresht drink describes geography landing...
1,132,led next day trees taking meat wool curdled mi...
2,132,enemies assert racial snobbery others hold cer...
3,132,may escaped professional observation two class...
4,132,nine o'clock gusty winter night stood lower st...


In [ ]:
print(dataset.isnull().sum())

author_id         0
processed_text    0
dtype: int64


In [ ]:
dataset.shape

(1825, 2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Distilbert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

44


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,3.763578,0.034247,0.008235,0.039535,0.011539
2,No log,3.476991,0.239726,0.150559,0.237597,0.147137
3,No log,2.832366,0.342466,0.277480,0.309690,0.248232
4,No log,2.245018,0.417808,0.387726,0.426744,0.356091
5,No log,1.851417,0.520548,0.481325,0.494961,0.451847
6,No log,1.647252,0.493151,0.456368,0.493798,0.435922
7,2.521200,1.481105,0.568493,0.531412,0.545833,0.513062
8,2.521200,1.416925,0.602740,0.613732,0.589535,0.562753


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=664, training_loss=2.0136714958282838, metrics={'train_runtime': 214.9215, 'train_samples_per_second': 48.911, 'train_steps_per_second': 3.09, 'total_flos': 1393540289593344.0, 'train_loss': 2.0136714958282838, 'epoch': 8.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.2158203 , -0.15588379, -1.1230469 , ..., -1.15625   ,
        -0.95654297, -1.4199219 ],
       [-1.3554688 , -1.828125  , -1.5957031 , ..., -1.3798828 ,
        -0.640625  , -0.86376953],
       [-0.63671875, -1.3574219 , -2.3964844 , ..., -1.5820312 ,
        -0.40771484, -1.3847656 ],
       ...,
       [-0.5996094 , -1.1035156 ,  2.1425781 , ...,  0.7988281 ,
        -2.5449219 , -0.93652344],
       [-0.41064453, -1.5654297 , -0.04971313, ...,  0.5102539 ,
        -1.6513672 , -1.875     ],
       [-0.83447266, -0.4729004 ,  0.61572266, ..., -1.2197266 ,
        -1.7304688 , -0.35375977]], dtype=float32), label_ids=array([27, 38, 29, 43,  2, 19, 39,  9, 18,  9, 11,  7, 27, 20, 16, 38, 10,
       12, 12,  3, 25, 32, 25, 43, 29, 14, 43, 29, 42, 39, 33,  0,  5, 33,
       12, 36, 15, 42, 29, 25, 26,  7, 19,  6, 39, 12, 16, 20, 10, 21, 18,
       30, 32, 35, 36, 12, 30,  1, 22, 40, 10, 40, 19, 21, 27, 24,  1, 26,
       16, 34, 15, 23, 38, 21, 2

## Experiment 2 - learning_rate = 5e-4

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-4,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,3.493579,0.109589,0.080946,0.098450,0.057869
2,No log,2.971635,0.171233,0.087356,0.183721,0.088572
3,No log,2.623733,0.239726,0.201357,0.206977,0.170216
4,No log,2.715814,0.226027,0.216025,0.225581,0.183328
5,No log,2.679756,0.226027,0.208511,0.234470,0.185185
6,No log,2.546468,0.308219,0.259876,0.303788,0.234628
7,2.614200,2.265166,0.315068,0.287163,0.354651,0.270042
8,2.614200,2.026234,0.397260,0.362348,0.398450,0.347459


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=664, training_loss=2.2656086151858412, metrics={'train_runtime': 236.4879, 'train_samples_per_second': 44.45, 'train_steps_per_second': 2.808, 'total_flos': 1393540289593344.0, 'train_loss': 2.2656086151858412, 'epoch': 8.0})

In [ ]:
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-2.1269531e+00,  3.3471680e-01, -1.4892578e+00, ...,
        -7.8320312e-01, -2.6308594e+00,  1.7528534e-03],
       [-2.5566406e+00, -2.9531250e+00, -2.0722656e+00, ...,
        -2.8730469e+00, -1.8481445e-01,  5.8447266e-01],
       [-1.1791992e-01, -3.4375000e+00, -2.7050781e+00, ...,
        -1.3154297e+00, -1.9736328e+00,  3.7060547e-01],
       ...,
       [-1.7402344e+00, -2.1406250e+00,  2.5136719e+00, ...,
        -2.1210938e+00, -3.0605469e+00, -8.6914062e-01],
       [-1.1437988e-01, -3.2636719e+00,  3.1308594e+00, ...,
        -2.5468750e+00, -2.3496094e+00, -2.6992188e+00],
       [-2.1445312e+00, -1.8974609e+00,  1.9199219e+00, ...,
        -2.8710938e+00, -2.5605469e+00, -4.7534180e-01]], dtype=float32), label_ids=array([27, 38, 29, 43,  2, 19, 39,  9, 18,  9, 11,  7, 27, 20, 16, 38, 10,
       12, 12,  3, 25, 32, 25, 43, 29, 14, 43, 29, 42, 39, 33,  0,  5, 33,
       12, 36, 15, 42, 29, 25, 26,  7, 19,  6, 39, 12, 16, 20, 10, 21, 18,

## Experiment 3 - learning_rate = 3e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=3e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,3.778735,0.068493,0.029153,0.046512,0.017651
2,No log,3.679554,0.178082,0.083899,0.176744,0.085851
3,No log,3.260368,0.273973,0.184022,0.244574,0.175211
4,No log,2.747779,0.315068,0.193600,0.303101,0.216520
5,No log,2.329005,0.424658,0.360161,0.423643,0.341423
6,No log,2.093610,0.506849,0.449430,0.496512,0.437640
7,2.969500,1.921186,0.541096,0.523671,0.529457,0.477786
8,2.969500,1.876935,0.554795,0.512458,0.536047,0.484305


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=664, training_loss=2.56738269759948, metrics={'train_runtime': 228.5157, 'train_samples_per_second': 46.001, 'train_steps_per_second': 2.906, 'total_flos': 1393540289593344.0, 'train_loss': 2.56738269759948, 'epoch': 8.0})

In [ ]:
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-1.1894531 ,  1.0195312 , -0.5678711 , ..., -1.1455078 ,
        -0.9746094 , -0.6430664 ],
       [-1.4023438 , -0.80908203, -0.88623047, ..., -1.5957031 ,
         0.21838379, -0.70458984],
       [ 0.21960449, -0.4645996 , -1.0273438 , ..., -1.3505859 ,
        -0.35888672, -1.0214844 ],
       ...,
       [-0.5776367 , -0.8466797 ,  1.5136719 , ...,  1.1259766 ,
        -0.9379883 , -0.13708496],
       [-0.65625   , -0.83935547,  0.7246094 , ..., -0.3215332 ,
        -0.49560547, -0.9067383 ],
       [-1.1025391 , -0.08581543,  1.1210938 , ..., -1.1064453 ,
        -1.0722656 , -0.7270508 ]], dtype=float32), label_ids=array([27, 38, 29, 43,  2, 19, 39,  9, 18,  9, 11,  7, 27, 20, 16, 38, 10,
       12, 12,  3, 25, 32, 25, 43, 29, 14, 43, 29, 42, 39, 33,  0,  5, 33,
       12, 36, 15, 42, 29, 25, 26,  7, 19,  6, 39, 12, 16, 20, 10, 21, 18,
       30, 32, 35, 36, 12, 30,  1, 22, 40, 10, 40, 19, 21, 27, 24,  1, 26,
       16, 34, 15, 23, 38, 21, 2

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,3.763097,0.027397,0.000651,0.023256,0.001266
2,No log,3.238134,0.301370,0.230040,0.296512,0.232040
3,No log,2.509746,0.376712,0.322130,0.393023,0.302572
4,No log,1.987804,0.486301,0.474557,0.492248,0.427150
5,No log,1.529943,0.568493,0.579070,0.568217,0.526615
6,No log,1.440549,0.616438,0.606118,0.610853,0.579346
7,2.214400,1.297737,0.650685,0.630814,0.625581,0.597472
8,2.214400,1.270898,0.671233,0.649945,0.650775,0.627665


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=664, training_loss=1.7087810010795135, metrics={'train_runtime': 219.3183, 'train_samples_per_second': 47.93, 'train_steps_per_second': 3.028, 'total_flos': 1393540289593344.0, 'train_loss': 1.7087810010795135, 'epoch': 8.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.9052734 ,  0.4970703 , -1.3876953 , ..., -2.9433594 ,
        -2.7070312 , -1.9833984 ],
       [-1.6074219 , -2.8398438 , -1.8583984 , ..., -2.5664062 ,
        -0.79833984, -1.2207031 ],
       [-0.9916992 , -1.921875  , -3.53125   , ..., -3.1953125 ,
        -1.9375    , -1.8623047 ],
       ...,
       [-1.5976562 , -2.9550781 ,  1.5830078 , ..., -0.83496094,
        -2.15625   , -0.65234375],
       [-1.3632812 , -2.7441406 , -0.3828125 , ..., -1.5898438 ,
        -1.3183594 , -2.2695312 ],
       [-2.1972656 , -1.7792969 ,  1.3398438 , ..., -3.0820312 ,
        -2.5507812 , -1.2539062 ]], dtype=float32), label_ids=array([27, 38, 29, 43,  2, 19, 39,  9, 18,  9, 11,  7, 27, 20, 16, 38, 10,
       12, 12,  3, 25, 32, 25, 43, 29, 14, 43, 29, 42, 39, 33,  0,  5, 33,
       12, 36, 15, 42, 29, 25, 26,  7, 19,  6, 39, 12, 16, 20, 10, 21, 18,
       30, 32, 35, 36, 12, 30,  1, 22, 40, 10, 40, 19, 21, 27, 24,  1, 26,
       16, 34, 15, 23, 38, 21, 2